# Example usage

To use `instrumentum` in a project:

In [1]:
import instrumentum

print(instrumentum.__version__)

0.2.0


In [2]:
import instrumentum.time_series as ts
import pandas as pd

data_file = "./sample_data/time_series.csv"
data_df = pd.read_csv(data_file)

ts.ts_print(data_df, col_target="ACQUIRED", col_key="CUSTOMER_KEY")

Total unique keys that with a possitive value:  2
Total records of ckeys that with a possitive value:  13
Total unique keys with no possitive value:  2
Total records of customers with no possitive values:  10


In [72]:
import instrumentum.features.stepwise as sw
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import logging

from instrumentum.utils.scorer_wrappers import optimizer_optuna_rf as rf, optimizer_optuna_xgb as xgb_

data_file = "./sample_data/simple.csv"
data_df = pd.read_csv(data_file)

data_df.loc[data_df['target'] == -1, 'target'] = 0

def myfunc2(X_train, y_train):
    ret, _ = rf(X_train, y_train,n_trials=12, n_repeats=1, verbose=False)
    return ret

def myfunc(X_train, y_train):

    model = DecisionTreeClassifier(random_state=0)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)
    return cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv).mean()

#logging.getLogger("")
sw.forward_stepwise(data_df.drop("target",axis=1)[['a', 'b', 'c', 'd', 'e', 'f']], data_df['target'], n_combs=1, rounding=4, add_always=False, verbose=logging.DEBUG, _scorer = myfunc2)


21-10-10 18:01 | INFO | Number of cores to be used: 8, total cores: 8

21-10-10 18:01 | INFO | Remaining columns to test: 6
21-10-10 18:01 | INFO | Combinations to test: 6
1    5003
0    4996
Name: target, dtype: int64
21-10-10 18:01 | DEBUG | Score 0.6757 for this combination: ['d']
21-10-10 18:01 | DEBUG | Score 0.8614 for this combination: ['a']
21-10-10 18:01 | DEBUG | Score 0.7699 for this combination: ['b']
21-10-10 18:01 | DEBUG | Score 0.6324 for this combination: ['f']
21-10-10 18:01 | DEBUG | Score 0.7214 for this combination: ['c']
21-10-10 18:01 | DEBUG | Score 0.6503 for this combination: ['e']
21-10-10 18:01 | INFO | Best score from combinations: 0.8614, global score 0
21-10-10 18:01 | INFO | Best score comes from columns: ['a']
21-10-10 18:01 | INFO | Best columns were added. All columns added so far ['a']

21-10-10 18:01 | INFO | Remaining columns to test: 5
21-10-10 18:01 | INFO | Combinations to test: 5
21-10-10 18:02 | DEBUG | Score 0.8386 for this combination: ['f']

KeyboardInterrupt: 

In [9]:
from instrumentum.utils.scorer_wrappers import optimizer_optuna_rf as rf, optimizer_optuna_xgb as xgb

rf(data_df[['a']], data_df['target'],n_trials=15, n_repeats=2, verbose=True)


Iteration:  0  Score:  0.7560299709409709
Iteration:  1  Score:  0.7563769664587665
Iteration:  2  Score:  0.779769069011669
Iteration:  3  Score:  0.7863203255106255
Iteration:  4  Score:  0.7558936977213977
Iteration:  5  Score:  0.7552819823722824
Iteration:  6  Score:  0.7865186296464296
Iteration:  7  Score:  0.7606478907051407
Iteration:  8  Score:  0.7545928727576229
Iteration:  9  Score:  0.7586741680895182
Iteration:  10  Score:  0.8388630885219885
Iteration:  11  Score:  0.8554777267051268
Iteration:  12  Score:  0.8618255170272171
Iteration:  13  Score:  0.8628325427503928
Iteration:  14  Score:  0.8272187739747741


(0.8628325427503928,
 RandomForestClassifier(max_depth=9.612430201103418, n_estimators=119))

In [11]:
rf(data_df[['a', 'b', 'c']], data_df['target'],n_trials=15, n_repeats=2, verbose=True)

Iteration:  0  Score:  0.8817018400685402
Iteration:  1  Score:  0.9041443745970745
Iteration:  2  Score:  0.8955893184343683
Iteration:  3  Score:  0.8890379322711823
Iteration:  4  Score:  0.8891576087892588
Iteration:  5  Score:  0.8895585561580062
Iteration:  6  Score:  0.8909287485158485
Iteration:  7  Score:  0.8872166206609707
Iteration:  8  Score:  0.9017355719835219
Iteration:  9  Score:  0.8888673473042473
Iteration:  10  Score:  0.9069727628480628
Iteration:  11  Score:  0.9071147960014458
Iteration:  12  Score:  0.9079464694672195
Iteration:  13  Score:  0.9060781544446044
Iteration:  14  Score:  0.8956785457634957


(0.9079464694672195,
 RandomForestClassifier(max_depth=9.300235249986745, n_estimators=115))

In [15]:
myfunc(data_df[['a', 'b', 'c']], data_df['target'])

0.8141794900571602

In [32]:
from itertools import product

a = [i for i, x in enumerate([False, True, False, True, True]) if x]


In [43]:
#data_df.iloc[:, a]
data_df.columns.get_loc('b')

1